# TensorflowとKerasでディープラーニング（犬と猫の2クラス認識）
tensorflowとkerasを用いてKaggleのDogs vs. Catsで提供されている犬と猫の画像のデータ(https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition )を学習させる。   
これまでのMNISTのようにあらかじめ準備されているデータセットではなく、画像で配布されているデータセットを用いるため、自分で画像を読み込んで学習させる必要がある。

## 1.必要なモジュールのimport
今回は画像の読み込みを行うためにkeras.preprocessing.imageからImageDataGeneratorを追加で読み込む。

In [1]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## 2.画像データの読み込み
読み込んだ画像データをニューラルネットワークに入れる時の設定をImageDataGeneratorで決めておく。
    
下では画像の0-255のピクセル値を計算の便宜上255で割って全ての数字を0-1の間に圧縮するように定義して、datagenと名前をつけて保存している。   
    
ここでは提示していないが、例えばhorizontal_flip=Trueと追加すると左右反転を自動で増やしたりしてくれる。他にも様々なデータの拡張が可能であり、詳細はhttps://keras.io/ja/preprocessing/image/ に説明がある。

In [2]:
datagen = ImageDataGenerator(rescale=1./255)

.flow_from_directoryという関数を用いて、ディレクトリを指定してそこから画像ファイルを読み込んで名前をつけておく。ここでは学習用データをtrain_generator、検証用データをvalidate_generatorと名付けている。    
    
読み込みの際にtarget_sizeで画像のサイズを指定する。画像は全て指定したサイズにリサイズされる。ここでは32x32ピクセルに指定している。     
    
batch_sizeはニューラルネットワークに入れる際に、いくつかのデータをまとめて読み込ませることになるため、学習または検証の際に一度に読み込むデータ数を指定する。ここでは20個を同時に読み込み、その誤差の平均を用いてネットワークのパラメータを更新していくように設定している。    
    
class_modeは"categorical"か"binary"か"sparse"か"None"のいずれか一つから選択する。ここではcategoricalとしているが、犬と猫の2クラス分類なのでbinaryでもよい。

In [3]:
train_generator = datagen.flow_from_directory(
        'dog_or_cat/train',
        target_size=(32, 32),
        batch_size=20,
        class_mode='categorical')
validate_generator = datagen.flow_from_directory(
        'dog_or_cat/validate',
        target_size=(32, 32),
        batch_size=20,
        class_mode='categorical')

Found 2000 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


## 3.Kerasを用いてニューラルネットを組む
    
Kerasを用いて畳み込みニューラルネットワークを組む。MNISTの時と同様のネットワークを用いているが、今回の画像は32x32ピクセルのカラー画像なので、input_shapeを(32,32,3)と変更している。また、2クラス分類問題なので最後のDense層のunit数を2に変更している。

In [4]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(2, activation='softmax'))

## 4.作成したニューラルネットをどのように学習させるかを設定する
.compile()で損失関数(loss)、最適化アルゴリズム(optimizer)、評価関数(metrics)を与えてニューラルネットを完成させる。    
    
分類問題なのでlossはcategorical_crossentropyを用い、確率的勾配降下法（SGD, Stochastic Gradient Descent）によるパラメータ更新としている。

In [5]:
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])

## 5.学習
学習を開始する。ImageDataGeneratorで作成したデータを学習させるときは、前回使用したmodel.fitではなく、model.fit_generatorを用いる。  
    
学習に用いるデータとしてtrain_generatorを指定し、検証に用いるデータとしてvalidate_generatorを指定する。   
   
steps_per_epochは1 epochあたりにデータを学習させる回数であり、通常は(データ総数/バッチ数)とする。ここでは学習データは2000個でバッチ数は20個なので、steps_per_epochは2000/20 = 100としている。   

validation_stepsも同様で、各エポックの終わりに精度を検証をするときにデータを入力する回数であり、通常は(データ総数/バッチ数)とする。ここでは検証用データは500個でバッチ数は20個なので、steps_per_epochは500/20 = 25としている。 
    
学習は10 epochsで設定しているが、おそらく時間は結構かかると思われる。（PCにもよるがCPUだと数分くらい）

In [6]:
model.fit_generator(train_generator, epochs=10, steps_per_epoch=100, validation_data=validate_generator, validation_steps = 25)

Epoch 1/10
100/100 [==============================] - 15s - loss: 0.6958 - acc: 0.5025 - val_loss: 0.6886 - val_acc: 0.5280
Epoch 2/10
100/100 [==============================] - 13s - loss: 0.6846 - acc: 0.5685 - val_loss: 0.6909 - val_acc: 0.5180
Epoch 3/10
100/100 [==============================] - 13s - loss: 0.6811 - acc: 0.5705 - val_loss: 0.6764 - val_acc: 0.6160
Epoch 4/10
100/100 [==============================] - 13s - loss: 0.6760 - acc: 0.5880 - val_loss: 0.6719 - val_acc: 0.6060
Epoch 5/10
100/100 [==============================] - 13s - loss: 0.6713 - acc: 0.5975 - val_loss: 0.6749 - val_acc: 0.5540
Epoch 6/10
100/100 [==============================] - 13s - loss: 0.6685 - acc: 0.6080 - val_loss: 0.6701 - val_acc: 0.5600
Epoch 7/10
100/100 [==============================] - 13s - loss: 0.6636 - acc: 0.6040 - val_loss: 0.6551 - val_acc: 0.6380
Epoch 8/10
100/100 [==============================] - 13s - loss: 0.6601 - acc: 0.6050 - val_loss: 0.6598 - val_acc: 0.6280
Epoch 9/

## 6.ネットワークを変えてやってみる
ニューラルネットワーク構築部分を変えて、精度が改善しないか試行する。

In [ ]:
model = Sequential()

#構造をここに記入

model.add(Dense(2, activation='softmax'))

In [ ]:
#モデルをコンパイルして学習させてみる
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=["accuracy"])
model.fit_generator(train_generator, epochs=10, steps_per_epoch=100, validation_data=validate_generator, validation_steps = 25)